In [58]:
import pandas as pd
import requests
import csv
import time
from bs4 import BeautifulSoup
from selenium import webdriver

In [81]:
# url = 'https://www.startbase.com/startups/?listOptions%5Bcompany-startup%5D=%7B%22version%22%3A1.3%2C%22sort%22%3A%22company.startbase_score%22%2C%22sortDirection%22%3A%22desc%22%2C%22display%22%3A%22small%22%2C%22itemsPerPage%22%3A200%2C%22page%22%3A1%2C%22userLocation%22%3Anull%2C%22filters%22%3A%7B%7D%7D'
# #headers = {'User-Agent': 'Mozilla/5.0'}
# #response = requests.get(url)#, headers=headers)
# # import urllib.request
# # with urllib.request.urlopen(url) as response:
# #    html = response.read()

# driver = webdriver.Firefox()
# driver.get(url)
# time.sleep(10)
# element = driver.find_element_by_class_name('main-body')
# html = element.get_attribute('innerHTML')

# soup = BeautifulSoup(html, 'html.parser')
# #driver.quit()


# all_as = soup.find_all("a")


# links = []
# for a in all_as:
#     link = a.get("href")
#     if link is not None and link.startswith('/organization/'):
#         links.append(link)  
        
# len(links)

50

In [3]:
# Get url
url = 'https://www.startbase.com/organization/w-o-wohnrechner-online/'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

NameError: name 'requests' is not defined

In [23]:
# Name, logo and teaser
title_card = soup.find('div', 'flex-grow-1 mx-lg-4')
startup_name = title_card.h1.get('data-name')
logo = title_card.h1.get('data-icon')
teaser = title_card.find('div', 'sb-teaser').text.strip()

In [24]:
# Body info
body_card = soup.find('ul', 'unformated row justify-content-start mb-0')
info_cards = body_card.find_all('li')
startup_info = []; labels_info = []

for count, info in enumerate(info_cards):
    labels_info.append(info.text.strip().split(':')[0])
    try:
        if count != 9: #10th item is links to social media
            info_piece = info.text.strip()
            # text cleaning
        #         if type(info_piece) == str: # When empty
        #             info_piece = ''
            if '\n ' in info_piece:
                info_piece = info_piece.split('\n ')[1]
            else:
                info_piece = info_piece.split(': ')[1]

            if info_piece[0] == ' ':
                info_piece = info_piece[1::]   
        else: #social media
            media_links = []
            for media in info.find_all('a'):
                media_links.append(media.get('href'))
            info_piece = media_links
    except:
        info_piece = ''

    startup_info.append(info_piece)

In [29]:
# About info
try:
    about_company = soup.find('div', {'class':'sb-card mb-4'}, id='company').p.text
except:
    about_company = ''

# Team info
try:
    # Get names
    member_names = soup.find_all('span',{'class':'name'})
    names = [];
    for name in member_names:
        names.append(name.text)
    # Get labels
    member_labels = soup.find_all('span',{'class':'label'})
    labels = [];
    for label in member_labels:
        labels.append(label.text) 

    team_members = dict(zip(names, labels))
except:
    team_members = '';

In [21]:
# Pool data in table
info_data = [startup_name] + [logo] + [teaser] + startup_info + [about_company] + [team_members]
info_labels = ['Name'] + ['Logo'] + ['Teaser'] + labels_info + ['About'] + ['Members']

company_data = dict(zip(info_labels, info_data))

startbase = pd.DataFrame([company_data])
startbase.head()

,Name,Logo,Teaser,Sector,Company Stage,Business Model,Customer Focus,Founding Year,Team Size,Handelsregister,Headquarter,Official Website,On Social Media,Legal Entity,About,Members
0,Noor Medical,https://cdn.startbase.com/images/company/noor-...,Provider of safe sterilization of medical inst...,Life Science & Healthcare,Seed Stage,Manufacturing,B2B,2017,1-10 Employees,Found,"Freiburg im Breisgau, DE",,"[https://www.instagram.com/noormedical/, https...",Noor Medical UG,"Über 1,5 Milliarden Menschen werden regelmäßig...",<zip object at 0x0000015040E63E88>
